In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [ ]:
def is_valid_index(array, index):
    return (index >= 0).all() and (index < array.shape).all()


def add_to_boxes(grid, visited, idx, box_idx):
    if is_valid_index(grid, idx):
        element = grid[*idx]
        if element in "[]" and not visited[*idx]:
            box_idx.add(tuple(int(x) for x in idx))
            visited[*idx] = True
            if element == "[":
                add_to_boxes(grid, visited, idx + [0, 1], box_idx)
            elif element == "]":
                add_to_boxes(grid, visited, idx + [0, -1], box_idx)
            add_to_boxes(grid, visited, idx + [-1, 0], box_idx)

In [ ]:
with open("data/day15/input.txt", "r") as file:
    init_raw = file.read()

In [ ]:
map_init_raw, directions_raw = init_raw.split("\n\n")

In [ ]:
directions = list(directions_raw.replace("\n", ""))

In [ ]:
direction_map = {k: n for n, k in enumerate(["^", ">", "v", "<"])}

In [ ]:
gps_factor = np.array([100, 1])

In [ ]:
char_to_int = {"#": 0, "@": 1, "O": 2, ".": 3, "[": 4, "]": 5}
char_to_img = np.vectorize(char_to_int.get)

In [ ]:
char_swap = {"[": "]", "]": "["}
flip = np.vectorize(lambda x: char_swap.get(x, x))

## Part 1

In [ ]:
map_init = np.array([list(x) for x in map_init_raw.split("\n")])

In [ ]:
map = map_init.copy()

for direction in tqdm(directions):

    map = np.rot90(map, k=direction_map[direction])
    idx_robot = np.argwhere(map == "@")[0]

    line_of_sight_full = np.flip(map[: idx_robot[0], idx_robot[1]])
    d_wall = np.argwhere(line_of_sight_full == "#").min()
    line_of_sight = line_of_sight_full[:d_wall]

    if any(np.isin(line_of_sight, ["."])):
        d_empty = np.argwhere(line_of_sight == ".").min()
        for i in range(d_empty + 1):
            line_of_sight[i] = "O"
        line_of_sight[0] = "@"
        map[*idx_robot] = "."

    map = np.rot90(map, k=4 - direction_map[direction])

plt.imshow(char_to_img(map))
plt.show()

In [ ]:
int(np.matmul(np.argwhere(map == "O"), gps_factor).sum())

## Part 2

In [ ]:
map_init_wide_raw = (
    map_init_raw.replace("#", "##")
    .replace("O", "[]")
    .replace(".", "..")
    .replace("@", "@.")
)

In [ ]:
map_init_wide = np.array([list(x) for x in map_init_wide_raw.split("\n")])

In [ ]:
map_wide = map_init_wide.copy()

for direction in tqdm(directions):

    map_wide = np.rot90(map_wide, k=direction_map[direction])
    idx_robot = np.argwhere(map_wide == "@")[0]

    if direction in "><":

        line_of_sight_full = np.flip(map_wide[: idx_robot[0], idx_robot[1]])
        d_wall = np.argwhere(line_of_sight_full == "#").min()
        line_of_sight = line_of_sight_full[:d_wall]

        if any(np.isin(line_of_sight, ["."])):
            d_empty = np.argwhere(line_of_sight == ".").min()
            for i in range(d_empty + 1):
                if direction == ">":
                    char = "]" if i % 2 == 0 else "["
                elif direction == "<":
                    char = "]" if i % 2 == 1 else "["
                line_of_sight[i] = char
            line_of_sight[0] = "@"
            map_wide[*idx_robot] = "."

    elif direction in "^v":

        if map_wide[*(idx_robot + [-1, 0])] != "#":

            if direction == "v":
                map_wide = flip(map_wide)

            visited = np.zeros(map_wide.shape, dtype=bool)
            box_idx = set()
            add_to_boxes(map_wide, visited, idx_robot + [-1, 0], box_idx)
            box_idx = np.array(list(box_idx))

            box_mask = np.ones(map_wide.shape, dtype=bool)
            box_mask_next = np.ones(map_wide.shape, dtype=bool)
            for i in box_idx:
                box_mask[*i] = False
                box_mask_next[*(i + [-1, 0])] = False
            map_wide_mask = np.ma.masked_array(map_wide.copy(), mask=box_mask)
            map_wide_mask_next = np.ma.masked_array(map_wide.copy(), mask=box_mask_next)

            if "#" not in np.ma.unique(map_wide_mask_next):
                for i in box_idx:
                    map_wide[*i] = "."
                for i in box_idx:
                    map_wide[*(i + [-1, 0])] = map_wide_mask[*i]
                map_wide[*(idx_robot + [-1, 0])] = "@"
                map_wide[*idx_robot] = "."

            if direction == "v":
                map_wide = flip(map_wide)

    map_wide = np.rot90(map_wide, k=4 - direction_map[direction])

plt.imshow(char_to_img(map_wide))
plt.show()

In [ ]:
int(np.matmul(np.argwhere(map_wide == "["), gps_factor).sum())